In [ ]:
import sys
import os

sys.path.append(os.path.abspath(".."))
import psycopg as psy
from tabulate import tabulate
from psycopg.rows import dict_row
import pandas as pd
from functions.conect_db import connect_to_db, connection_string
from functions.create_tables import create_tables
from functions.create_indexes import create_all_indexes
from functions.create_triggers import create_all_triggers
from functions.drop_tables import drop_tables, table_names_from_sql
from functions.inserts import insert_generic

### Conectando ao banco de dados

In [ ]:
conn_info = connect_to_db()
connection_string = connection_string()
print(connection_string)

### Criando tabelas

In [ ]:
file = '../sql/create_tables.sql'
tables_created = create_tables(conn_info, file, 'public')
print('Created table(s):')
for table in tables_created:
    print(f" - {table}")

### Dropando tabelas

In [ ]:
tables = table_names_from_sql('../sql/create_tables.sql')

print(f'Dropped table(s):')
for name in reversed(tables):
    drop_tables(conn_info, [name])
    print(f" - {name}")
print()
print(f'Dropped {len(tables)} tables successfully')

### Criando Trigger para Auditoria

In [ ]:
file_func = r"C:\Users\felip\OneDrive\git_work\smart-city-os\sql\trigger_functions.sql"
file_trig = r"C:\Users\felip\OneDrive\git_work\smart-city-os\sql\triggers.sql"
create_all_triggers(conn_info, file_func, file_trig)

### Criando Índices

In [ ]:
file_in = r"C:\Users\felip\OneDrive\git_work\smart-city-os\sql\index.sql"
create_all_indexes(conn_info, file_in)

### Inserts de Teste

In [ ]:
insert_generic(conn_info,'app_user','public',('Paul', 'Doey', '12345678904', '1990-01-01', 'paul.doey@example.com', '12345678903', '123456 Main St', 'pauldoe', 'password_hash'),['id','first_name','email'])

In [ ]:
def query_db(conn_info, query):
    query_upper = query.strip().upper()
    if not query_upper.startswith("SELECT"):
        raise ValueError("Only SELECT queries are allowed")
    with psy.connect(conn_info, row_factory=dict_row) as conn:
        with conn.cursor() as cur:
            cur.execute(query)
            rows = cur.fetchall()
            return rows
query = "SELECT * FROM app_user;"
try:
    result = query_db(conn_info, query)
except Exception as e:
    print(f"Error: {e}")
    result = []
#print(result)

print(tabulate(result, headers="keys", tablefmt="github"))

In [ ]:
def query_to_df(query, connection_string):
   query_upper = query.strip().upper()
   if not query_upper.startswith("SELECT"):
      raise ValueError("Only SELECT queries are allowed")
   df = pd.read_sql(query, connection_string)
   return df

query = "SELECT new_values FROM audit_log"
df = query_to_df(query, connection_string)
print(dict(df['new_values'][0]))
os.makedirs("../csv", exist_ok=True)
df.to_csv("../csv/audit_log.csv", index=False)